In [3]:
import snap
import numpy as np

In [276]:
class LouvainGraph(object):
    """
    Weighted, undirected graph for Louvain Method. Cannot change the topology of graph once it is initialized. 
    The Community can be changed after the graph is initialized
    """
    def __init__(self, Edges, Communities=None):
        self.Edges = np.array(Edges, dtype=int)  # adjacency matrix
        self.n = self.Edges.shape[0]  # number of nodes
        self.two_m = np.sum(self.Edges)  # number of edges * 2
        if Communities is not None:
            self.Communities = np.array(Communities, dtype=int)  # each node belongs to which community
        else:
            self.Communities = np.arange(self.n)  # initialize with identity community, if no community is given
            
        self.Degrees = np.sum(self.Edges, axis=0)  # degree distribution
        self.ExpectedEdges = self.Degrees.reshape((-1, 1)) * self.Degrees.reshape((1, -1)) / self.two_m  # expected edges matrix
        
    def __str__(self):
        return "Edges:\n%s\nCommunities:\n%s"%(self.Edges, self.Communities)
    
    def rearrange_community(self):
        Communities_set = np.unique(self.Communities)
        mapper = {}
        # print(Communities_set)
        for index, value in enumerate(Communities_set):
            # print(index, value)
            mapper[value] = index
        for index, value in enumerate(self.Communities):
            self.Communities[index] = mapper[value]
        
    def neighbors_of_node(self, node):
        nodes = (self.Edges[node] != 0)
        nodes[node] = False  # exclude the inquired node itself
        return np.where(nodes)[0]
    
    def community_of_node(self, node):
        return self.Communities[node]
    
    def nodes_in_community(self, community):
        return np.where(self.Communities == community)[0]
    
    def nodes_in_same_community_with_node(self, node):
        community = self.community_of_node(node)
        nodes = (self.Communities == community)
        nodes[node] = False  # exclude the inquired node itself
        return np.where(nodes)[0]
        
    def modularity(self):
        SameCommunityMask = self.Communities.reshape((-1, 1)) == self.Communities.reshape((1, -1))
        return np.sum((self.Edges - self.ExpectedEdges)[SameCommunityMask]) / self.two_m
    
    def modularity_gain_from_isolating(self, node):
        nodes_c = self.nodes_in_same_community_with_node(node)
        return - 2 * np.sum((self.Edges - self.ExpectedEdges)[node, nodes_c]) / self.two_m
    
    def modularity_gain_from_merging(self, node, community):
        nodes_c = self.nodes_in_community(community)
        return 2 * np.sum((self.Edges - self.ExpectedEdges)[node, nodes_c]) / self.two_m
    
    def modularity_optimization(self, maxIter=5, random=False, seed=0):
        """
        Phase 1: modularity optimization
        """
        self.rearrange_community()
        if random:
            np.random.seed(seed)
        for epoch in range(maxIter):
            change = False
            if random:
                nodes = np.random.choice(self.n, self.n, False)
            else:
                nodes = range(self.n)
            for node in nodes:
                # print('see node', node)
                modularity_gain_from_isolating = self.modularity_gain_from_isolating(node)
                neighbors = self.neighbors_of_node(node)
                # print('neighbors', neighbors)
                
                max_modularity_gain_from_merging = 0
                community_to_join = -1
                for neighbor in neighbors:
                    community = self.community_of_node(neighbor)
                    modularity_gain_from_merging = self.modularity_gain_from_merging(node, community)
                    if modularity_gain_from_merging > max_modularity_gain_from_merging:
                        max_modularity_gain_from_merging = modularity_gain_from_merging
                        community_to_join = community
                # print(modularity_gain_from_isolating)
                # print(max_modularity_gain_from_merging, community_to_join)
                if modularity_gain_from_isolating > 0 and max_modularity_gain_from_merging <= 0:
                    # only isolating! 
                    # print('isolating')
                    self.Communities[node] = np.max(self.Communities) + 1
                    change = True
                elif modularity_gain_from_isolating + max_modularity_gain_from_merging > 0:
                    # also max_modularity_gain_from_merging > 0
                    # isolating and merging! 
                    # print('changing')
                    # print(self.Communities)
                    self.Communities[node] = community_to_join
                    change = True
                    # print(self.Communities)
                else:
                    # do nothing
                    # print('do nothing')
                    pass
            
            if not change:
                break
        self.rearrange_community()
        return self.modularity()
    
    def community_aggregation(self):
        """
        Phase 2: community aggregation
        """
        self.rearrange_community()
        n_c = np.max(self.Communities) + 1
        if n_c == self.n:
            return None, None, False
        Edges = np.zeros((n_c, n_c), dtype=int)
        for i in range(self.n):
            for j in range(self.n):
                c_i, c_j = self.community_of_node(i), self.community_of_node(j)
                Edges[c_i, c_j] += self.Edges[i, j]
        return Edges, self.Communities, True

In [234]:
def genGraph(C=4):
    Edges_cluster = np.ones((4, 4), dtype=int) - np.eye(4, dtype=int)
    Edges = np.zeros((4*C, 4*C), dtype=int)
    for i in range(C):
        Edges[4*i:4*i+4, 4*i:4*i+4] = Edges_cluster
        Edges[4*i+3, (4*i+4)%(4*C)] = 1
        Edges[(4*i+4)%(4*C), 4*i+3] = 1
    return Edges

def genBlondelGraph():
    Edges = np.zeros((16, 16), dtype=int)
    Edges[0, (2, 3, 4, 5)] = 1
    Edges[1, (2, 4, 7)] = 1
    Edges[2, (0, 1, 4, 5, 6)] = 1
    Edges[3, (0, 7)] = 1
    Edges[4, (0, 1, 2, 10)] = 1
    Edges[5, (0, 2, 7, 11)] = 1
    Edges[6, (2, 7, 11)] = 1
    Edges[7, (1, 3, 5, 6)] = 1
    Edges[8, (9, 10, 11, 14, 15)] = 1
    Edges[9, (8, 12, 14)] = 1
    Edges[10, (4, 8, 11, 12, 13, 14)] = 1
    Edges[11, (5, 6, 8, 10, 13)] = 1
    Edges[12, (9, 10)] = 1
    Edges[13, (10, 11)] = 1
    Edges[14, (8, 9, 10)] = 1
    Edges[15, (8)] = 1
    return Edges

In [260]:
Gsmall = genGraph(4)
Gbig = genGraph(32)
Gblondel = genBlondelGraph()

## 3.2 Louvain algorithm on a 16 node network

In [281]:
EdgesStack = [Gsmall]
CommunitiesStack = []
ModularityStack = []
while True:
    Edges = EdgesStack[-1]
    Graph = LouvainGraph(Edges)
    modularity = Graph.modularity_optimization(20)
    Edges, Communities, new_graph_flag = Graph.community_aggregation()
    if not new_graph_flag:
        break
    EdgesStack.append(Edges)
    CommunitiesStack.append(Communities)
    ModularityStack.append(modularity)

print('Final Community Hierarchy, fine to coarse:')
Community = CommunitiesStack[0]
CommunitieHierarchy = [Community]
print(Community, ModularityStack[0])
for i in range(1, len(CommunitiesStack)):
    Community = CommunitiesStack[i][Community]
    CommunitieHierarchy.append(Community)
    print(Community, ModularityStack[i])

Final Community Hierarchy, fine to coarse:
[0 0 0 0 1 1 1 1 2 2 2 2 3 3 3 3] 0.6071428571428571


In [283]:
EdgesStack

[array([[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]]),
 array([[12,  1,  0,  1],
        [ 1, 12,  1,  0],
        [ 0,  1, 1

- The weight of any edge between two distinct nodes in $H$ is 1. 
- The weight of any self.edge in $H$ is 12. 
- The modularity of $H$ is 0.607. 

In [289]:
J = LouvainGraph(EdgesStack[-1], [0, 0, 1, 1])
Edges, Communities, flag = J.community_aggregation()
J = LouvainGraph(Edges)
print(J)
print(J.modularity())

Edges:
[[26  2]
 [ 2 26]]
Communities:
[0 1]
0.42857142857142855


- The weight of any edge between two distinct nodes in $J$ is 2. 
- The weight of any self.edge in $J$ is 26. 
- The modularity of $H$ is 0.429. 

## 3.3 Louvain algorithm on a 128 node network

In [290]:
EdgesStack = [Gbig]
CommunitiesStack = []
ModularityStack = []
while True:
    Edges = EdgesStack[-1]
    Graph = LouvainGraph(Edges)
    modularity = Graph.modularity_optimization(20)
    Edges, Communities, new_graph_flag = Graph.community_aggregation()
    if not new_graph_flag:
        break
    EdgesStack.append(Edges)
    CommunitiesStack.append(Communities)
    ModularityStack.append(modularity)

print('Final Community Hierarchy, fine to coarse:')
Community = CommunitiesStack[0]
CommunitieHierarchy = [Community]
print(Community, ModularityStack[0])
for i in range(1, len(CommunitiesStack)):
    Community = CommunitiesStack[i][Community]
    CommunitieHierarchy.append(Community)
    print(Community, ModularityStack[i])

Final Community Hierarchy, fine to coarse:
[ 0  0  0  0  1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5
  6  6  6  6  7  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10 11 11 11 11
 12 12 12 12 13 13 13 13 14 14 14 14 15 15 15 15 16 16 16 16 17 17 17 17
 18 18 18 18 19 19 19 19 20 20 20 20 21 21 21 21 22 22 22 22 23 23 23 23
 24 24 24 24 25 25 25 25 26 26 26 26 27 27 27 27 28 28 28 28 29 29 29 29
 30 30 30 30 31 31 31 31] 0.8258928571428571
[ 0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2
  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8
  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14
 15 15 15 15 15 15 15 15] 0.8660714285714286


In [291]:
EdgesStack[1]

array([[12,  1,  0, ...,  0,  0,  1],
       [ 1, 12,  1, ...,  0,  0,  0],
       [ 0,  1, 12, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 12,  1,  0],
       [ 0,  0,  0, ...,  1, 12,  1],
       [ 1,  0,  0, ...,  0,  1, 12]])

- The weight of any edge between two distinct nodes in $H_{big}$ is 1. 
- The weight of any self.edge in $H_{big}$ is 12. 
- The modularity of $H_{big}$ is 0.826. 

In [292]:
EdgesStack[2]

array([[26,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 1, 26,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1, 26,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1, 26,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1, 26,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1, 26,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1, 26,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1, 26,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1, 26,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1, 26,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 26,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 26,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 26,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

- The weight of any edge between two distinct nodes in $J_{big}$ is 1. 
- The weight of any self.edge in $J_{big}$ is 26. 
- The modularity of $J_{big}$ is 0.866. 

区别在于每一个2\*2方块非对角线的元素
随着图的尺寸增大， Adjacency Matrix中非self loop对modularity的贡献是1/n，但expected对modularity的贡献是是1/n^2  
当2m足够大的时候，就有adjacency matrix的对应元素大于expected，因此加入这些元素可以提升modularity

1/2m v.s. d_i d_j / 2m^2